In [1]:
import pandas as pd
import re
import sqlite3
import datetime
import os
from openpyxl import Workbook, load_workbook

In [13]:
content.columns

Index(['SkipPause[True,18,0]', 'MethodName[True,239,1]',
       'SampleName[True,108,2]', 'SampleAmount[False,78,3]',
       'SampleDescription[False,120,4]', 'SampleId[False,90,5]',
       'InjectionYesNo[False,72,6]', 'PeakInformation[False,104,7]',
       'SampleInjection[False,138,8]', 'NOTES_STRING[False,68,9]',
       '#Sample Well[True,109,10]', '#Plate_Sample[True,115,13]'],
      dtype='object')

In [54]:

for fi in os.listdir(fp):
    if re.search('^\d+', fi):
        content = pd.read_csv(f'{fp}{fi}', sep='\t') 
        fi = fi.split('_')[2].split('.')[0]
        with open(f"{fp}gilson_set_file_{fi}.txt", 'w') as f:
            f.write('Sample_Name\tTube_Barcode\tBrooks_Barcode\n')

            for i,r in content.iterrows():
                 if i > 0 and re.search('^[L|M|C]', r['SampleName[True,108,2]']):
                        #print(r['SampleName[True,108,2]'],'\t',r['NOTES_STRING[False,68,9]'],'\t', r['SampleDescription[False,120,4]'])
                        f.write(f"{r['SampleName[True,108,2]']}\t{r['SampleDescription[False,120,4]']}\t{r['NOTES_STRING[False,68,9]']}\n")

In [2]:
fp = '/users/trinhsk/documents/pythonscripts/Gilson Worklist Generator/'

conn = sqlite3.connect(f'{fp}gilsontsl.db')
c = conn.cursor()

In [67]:
conn.close()

In [6]:
sqlite_create_table_query = '''CREATE TABLE GILSON_SET_TABLE 
                            (RUN_DATE TIMESTAMP,
                            ID TEXT NOT NULL, 
                            SAMPLE_NUMBER INTEGER NOT NULL,
                            SAMPLE_NAME TEXT NOT NULL, 
                            TUBE_BARCODE TEXT PRIMARY KEY, 
                            BROOKS_BARCODE TEXT NOT NULL);'''

sqlite_delete_table_query = '''DROP TABLE GILSON_SET_TABLE;'''

#c.execute(sqlite_delete_table_query)
c.execute(sqlite_create_table_query)

In [11]:
for fi in os.listdir(fp):
    if re.search('^\d+', fi):
        split_fi_name = fi.split('_')
        suffix = split_fi_name[2].split('.')[0]
        input_file = f"{fp}gilson_set_file_{suffix}.txt"
        fi_content = pd.read_csv(input_file, sep='\t')
        num_rows = len(fi_content.index)
        for i,r in fi_content.iterrows():
            numbr = i % num_rows
            ls_tp_content = [(datetime.datetime.now(), split_fi_name[0], numbr, r['Sample_Name'], r['Tube_Barcode'], str(int(r['Brooks_Barcode'])))]
            c.executemany("INSERT INTO GILSON_SET_TABLE VALUES (?,?,?,?,?,?)", ls_tp_content)
            
        conn.commit()

In [50]:
wb_load = load_workbook(f"{fp}excel_gilson_script_test.xlsx")
print(wb_load.sheetnames)

['Sheet1', 'Set1', 'Set2']


In [51]:
wset1 = wb_load['Set1']
wset2 = wb_load['Set2']

In [52]:
ids = c.execute('''SELECT DISTINCT ID FROM GILSON_SET_TABLE''').fetchall()
for rw in ids:
    print(rw)

('1520030001',)
('1520030002',)
('1520030003',)


In [53]:
project_num = '1520030001'

for idt in ids:
    if project_num in idt:
        row_data = c.execute(f'''SELECT * FROM GILSON_SET_TABLE WHERE ID = {idt[0]}''').fetchall()

#for i,row in enumerate(row_data):
#    print(row)

In [54]:
header = ['RUN_DATE', 'ID', 'SAMPLE_NUMBER', 'SAMPLE_NAME', 'TUBE_BARCODE', 'BROOKS_BARCODE']
for i in range(1,7):
    #print(chr(64+i))
    wset1[f"{chr(64+i)}1"] = header[i-1]

In [55]:
rcnt=0
ccnt=0
for row in range(2, len(row_data)+1):
    for col in range(1,7):
        _ = wset1.cell(column=col, row=row, value=f"{row_data[rcnt][ccnt%6]}")
        #print(row_data[rcnt][ccnt%6])
        #print(f"{ccnt%6} {rcnt}")
        ccnt+=1
    rcnt+=1

In [19]:
#for i in range(3,len(row_data)):
#    wset1.append(row_data[i])
    #print(row_data[i])

In [67]:
#wset1['A10'].value

In [64]:
def update_set_data(project_id, set_num, excel_file = 'excel_gilson_script_test'):
    wb_load = load_workbook(f"{fp}{excel_file}.xlsx")
    wset = wb_load[f"Set{set_num}"]
    row_data = c.execute(f'''SELECT * FROM GILSON_SET_TABLE WHERE ID = {project_id}''').fetchall()
    brooks_bc = c.execute(f'''SELECT DISTINCT BROOKS_BARCODE FROM GILSON_SET_TABLE WHERE ID = {project_id}''').fetchone()
    #clear content of sheet
    for row in wset['A1:H24']:
        for cell in row:
            cell.value = None
    #add header to sheet
    for i in range(1,7):
        wset[f"{chr(64+i)}1"] = header[i-1]
    #populate the cells
    rcnt=0
    ccnt=0
    for row in range(2, len(row_data)+1):
        for col in range(1,7):
            _ = wset.cell(column=col, row=row, value=f"{row_data[rcnt][ccnt%6]}")
            ccnt+=1
        rcnt+=1
    #overwrite the excel file
    wb_load.save(f'{fp}{excel_file}.xlsx')
    print('completed update')
    
def save_tsl_file(pid_1, pid_2, brooks_bc1, brooks_bc2):
    #save new tab-delimited file to directory
    parent_dir = '/Volumes/npsg/tecan/SourceData/SecondStage/Completed Projects Injections/'
    pid = pid_1[:8]
    pid_suffix1 = pid_1[8:]
    pid_suffix2 = pid_2[8:]
    for fi in os.listdir(parent_dir):
        if fi.startswith(pid) and os.path.isdir(os.path.join(parent_dir,fi)):
            file_path = f'{parent_dir}/3_SecondStage Sample List/{pid}_{pid_suffix1}_{pid_suffix2}_{brooks_bc1}_{brooks_bc2}.tsl'
    with open(file_name, 'w') as fw:
        wf.write('\n')

In [65]:
update_set_data('1520030002',1)

134
